In [ ]:
# This notebook is based on the exoeriemnt presented in the file volume-preserving-experiment.ipynb
# Of the repository https://github.com/vislearn/Coupling-Universality
# Authored by F.Draxler and S.Wahl

In [ ]:
import matplotlib.pyplot as plt
import torch 
from torch import Tensor
import torch.distributions as D
from FrEIA.utils import force_to
from functools import partial
import torch.nn as nn
import FrEIA.modules as Fm
import FrEIA.framework as Ff
from typing import Iterable, Tuple, Callable,Any
from FrEIA.modules.base import ShapeList
import numpy as np
import tqdm
import os

from pinf.models.GMM import GMM
from pinf.plot.utils import eval_pdf_on_grid_2D

Settings

---

In [ ]:
torch.manual_seed(47)
device = "cpu"
n_layers = 15
res = 500
lim = 1.25
lim_zoom_in = 0.25
lr= 1e-3
batch_size = 128
milestones = [5000,10000,15000]
gamma = 0.1
n_batches = 25000
save_freq = 5000
train_new = False

folder = "../../results/visualization_2D_GMM_vp/"

Model definition

---

In [ ]:
#Modified GIN coupling block to allow variable jacobian determinant
class ModifiedGINCouplingBlock(Fm.GINCouplingBlock):
    def __init__(self, dims_in, dims_c=[], subnet_constructor: Callable[..., Any] = None, clamp: float = 2, clamp_activation: str | Callable[..., Any] = "ATAN", split_len: float | int = 0.5,normalize:bool = True):
        '''
        Additional parameters:
            normalize:      Return constant Jacobian determinant if true
        '''

        super().__init__(dims_in, dims_c, subnet_constructor, clamp, clamp_activation, split_len)
        self.normalize = normalize

    def _coupling1(self, x1, u2, rev=False):

        # notation (same for _coupling2):
        # x: inputs (i.e. 'x-side' when rev is False, 'z-side' when rev is True)
        # y: outputs (same scheme)
        # *_c: variables with condition appended
        # *1, *2: left half, right half
        # a: all affine coefficients
        # s, t: multiplicative and additive coefficients
        # j: log det Jacobian

        a2 = self.subnet2(u2)
        s2, t2 = a2[:, :self.split_len1], a2[:, self.split_len1:]
        s2 = self.clamp * self.f_clamp(s2)

        #Constant Jacobian determinant of one
        if self.normalize: 
            s2 = s2 - s2.mean(1, keepdim=True)
            jac = 0.0

        #Variable Jacobian determinant
        else:
            jac = s2.sum(-1)

        if rev:
            y1 = (x1 - t2) * torch.exp(-s2)
            return y1, jac
        else:
            y1 = torch.exp(s2) * x1 + t2
            return y1, jac
        
    def _coupling2(self, x2, u1, rev=False):
        a1 = self.subnet1(u1)
        s1, t1 = a1[:, :self.split_len2], a1[:, self.split_len2:]
        s1 = self.clamp * self.f_clamp(s1)

        #Constant Jacobian determinant of one
        if self.normalize: 
            s1 = s1- s1.mean(1, keepdim=True)
            jac = 0.0

        #Variable Jacobian determinant
        else:
            jac = s1.sum(-1)

        if rev:
            y2 = (x2 - t1) * torch.exp(-s1)
            return y2, -jac
        else:
            y2 = torch.exp(s1) * x2 + t1
            return y2, jac
        
#Global scaling block for INN with constant Jacobian determinant
class ScalingBlock(Fm.InvertibleModule):
    def __init__(self, dims_in: ShapeList, dims_c: ShapeList = None):
        super().__init__(dims_in, dims_c)

        #Learnable scaling parameter
        self.a = nn.Parameter(torch.ones([1]))

    def output_dims(self, input_dims: ShapeList) -> ShapeList:
        return input_dims
    
    def forward(self, x_or_z: Iterable[Tensor], c: Iterable[Tensor] = None, rev: bool = False, jac: bool = True) -> Tuple[Tuple[Tensor], Tensor]:
        
        x = x_or_z[0]
        N= x.shape[0]
        d = x.shape[1]

        if rev:
            jac = - d * torch.log(self.a)
            x = x / self.a

        else:
            jac = + d * torch.log(self.a)
            x = x * self.a

        return ((x,),jac)     

#Construct the subnetworks for the normalizing flows
def get_subnet(c_in,c_out):

    d_hidden = 128
    layers = nn.Sequential(
        nn.Linear(c_in,d_hidden),
        nn.ReLU(),
        nn.Linear(d_hidden,d_hidden),
        nn.ReLU(),
        nn.Linear(d_hidden,d_hidden),
        nn.ReLU(),
        nn.Linear(d_hidden,c_out)
    )

    #Initialize the weights of the linear layers
    for layer in layers:
        if isinstance(layer,nn.Linear):
            nn.init.xavier_normal_(layer.weight)

    #Set the weights and the bias of the final layer to zero
    layers[-1].weight.data.fill_(0.0)
    layers[-1].bias.data.fill_(0.0)

    return layers

Initialize the data distribution

---

In [ ]:
m1 = torch.tensor([-0.5,-0.5]).reshape(1,-1)
m2 = torch.tensor([0.5,0.5]).reshape(1,-1)
means = torch.cat((m1,m2),0)

S1 = (torch.eye(2) * 0.2).reshape(1,2,2)
S2= (torch.eye(2) * 0.1).reshape(1,2,2)
S = torch.cat((S1,S2),0)
p_GMM = GMM(means = means,covs=S,weights = torch.tensor([0.5,0.5]))

Train the normalizing flow with constant Jacobian determinant

---

In [ ]:
INN_const_jac = Ff.SequenceINN(2)

for i in range(n_layers):
    INN_const_jac.append(module_class = ModifiedGINCouplingBlock,subnet_constructor = get_subnet,normalize = True)
INN_const_jac.append(ScalingBlock)

INN_const_jac.to(device)

In [ ]:
#Training of an INN
def train(INN,p_data:Callable,device:str,lr:float,milestones:list,gamma:float,batch_size:int,n_batches:int,experiment_name:str,save_freq:int):
    """
    parameters:
        INN:                Normalizing flow to train
        p_data:             Function to get samples following the target distribution
        lr:                 Learning rate
        milestones:         Milestones for learning rate decay
        gamma:              Factor for learning rate decay
        batch_size:         Bacth size
        n_batches:          Number of batches
        experiment_name:    Name of the training run
        save_freq:          Frequency of saving the state dict
    """


    INN.train()

    #Create a folder for the training results
    if not os.path.exists(os.path.join(folder,experiment_name)):
        os.makedirs(os.path.join(folder,experiment_name))

    #Latent distribution of the model
    p_0 = force_to(D.MultivariateNormal(torch.zeros(2),torch.eye(2)),device)
    
    #Initialize the optimizer and the lr scheduler
    optimizer = torch.optim.Adam(INN.parameters(),lr = lr)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer,milestones=milestones,gamma =gamma)

    #Storage
    loss_storage = torch.zeros(n_batches)
    jacobian_storage = torch.zeros([n_batches,2])

    #Train the model
    for i in tqdm.tqdm(range(n_batches)):
        
        #Get training data
        x = p_data.sample(N = batch_size).to(device)

        #Compute the objective
        z,jac = INN(x)
        nll = - (p_0.log_prob(z) + jac).mean()

        #Optimize
        optimizer.zero_grad()
        nll.backward()
        optimizer.step()

        lr_scheduler.step()

        #Store the results
        loss_storage[i] = nll.item()
        jacobian_storage[i][0] = jac.mean()
        jacobian_storage[i][1] = jac.std().item()

        #Store the model
        if ((i+1) % save_freq) == 0:
            torch.save(INN.state_dict(),folder + f"{experiment_name}/state-dict_iteration-{i+1}.pt")

    
    #Save the recorded data
    np.savetxt(folder +f"{experiment_name}/loss.txt",loss_storage.cpu().detach().numpy())
    np.savetxt(folder +f"{experiment_name}/jac.txt",jacobian_storage.cpu().detach().numpy())

In [ ]:
if train_new:    
    train(
        p_data = p_GMM,
        INN = INN_const_jac,
        device = device,
        lr = lr,
        milestones = milestones,
        gamma = gamma,
        batch_size = batch_size,
        n_batches = n_batches,
        experiment_name =  "const_jac_GMM_run-0",
        save_freq = save_freq
        )

Train the normalizing flow with variable Jacobian determinant

---

In [ ]:
INN_variable_jac = Ff.SequenceINN(2)

for i in range(n_layers):
    INN_variable_jac.append(module_class = ModifiedGINCouplingBlock,subnet_constructor = get_subnet,normalize = False)
  
INN_variable_jac.to(device)

In [ ]:
if train_new:
    train(
        p_data = p_GMM,
        INN = INN_variable_jac,
        device = device,
        lr = lr,
        milestones = milestones,
        gamma = gamma,
        batch_size = batch_size,
        n_batches = n_batches,
        experiment_name =  "variable_jac_GMM_run-0",
        save_freq = save_freq
        )

Load the trained models

---

In [ ]:
INN_const_jac.load_state_dict(torch.load(os.path.join(folder,f"const_jac_GMM_run-0/state-dict_iteration-{n_batches}.pt")))
INN_const_jac.eval()

In [ ]:
INN_variable_jac.load_state_dict(torch.load(os.path.join(folder,f"variable_jac_GMM_run-0/state-dict_iteration-{n_batches}.pt")))
INN_variable_jac.eval()

Visualize the volume change of the different models

---

In [ ]:
#Number of cells per dimension
n_cells = 10
n_res_cell = 200
limLatent = 3

#Get the area of a cell in the data space 
def get_transformed_Area(z_1_lims,z_2_lims,INN,res):

    z_1_grid_ = torch.linspace(z_1_lims[0],z_1_lims[1],res)
    z_2_grid_ = torch.linspace(z_2_lims[0],z_2_lims[1],res)

    z_1_grid,z_2_grid = torch.meshgrid(z_1_grid_,z_2_grid_)

    z_1_A = z_1_grid[1:,:-1]
    z_2_A = z_2_grid[1:,:-1]
                    
    z_1_B = z_1_grid[1:,1:]
    z_2_B = z_2_grid[1:,1:]

    z_1_C = z_1_grid[:-1,1:]
    z_2_C = z_2_grid[:-1,1:]

    z_1_D = z_1_grid[:-1,:-1]
    z_2_D = z_2_grid[:-1,:-1]

    z_A = torch.cat((z_1_A.reshape(-1,1),z_2_A.reshape(-1,1)),1)
    z_B = torch.cat((z_1_B.reshape(-1,1),z_2_B.reshape(-1,1)),1)
    z_C = torch.cat((z_1_C.reshape(-1,1),z_2_C.reshape(-1,1)),1)
    z_D = torch.cat((z_1_D.reshape(-1,1),z_2_D.reshape(-1,1)),1)

    #Get the latent area
    A_latent = 0.5 * ((z_1_A - z_1_C) * (z_2_B - z_2_D) + (z_2_A - z_2_C) * (z_1_D - z_1_B)).reshape(-1)

    with torch.no_grad():
        x_A = INN(z_A,rev=True)[0].detach().cpu()
        x_B = INN(z_B,rev=True)[0].detach().cpu()
        x_C = INN(z_C,rev=True)[0].detach().cpu()
        x_D = INN(z_D,rev=True)[0].detach().cpu()

    A_data  = 0.5 * ((x_A[:,0] - x_C[:,0]) * (x_B[:,1] - x_D[:,1]) + (x_D[:,0] - x_B[:,0]) * (x_A[:,1] - x_C[:,1])).abs() 

    #Get the centre of the polygons in the data space
    x_centre = 0.25 * (x_A + x_B + x_C + x_D)

    #Check if the polygon is convex
    def cross_product(a, b, c):
        """
        Berechnet das Kreuzprodukt der Vektoren AB und AC.
        """
        ab = a-b  # Vektor AB
        ac = a-c  # Vektor AC
        return ab[:,0] * ac[:,1] - ab[:,1] * ac[:,0]

    s1 = cross_product(x_A,x_B,x_D)
    s2 = cross_product(x_B,x_C,x_A)
    s3 = cross_product(x_C,x_D,x_B)
    s4 = cross_product(x_D,x_A,x_C)

    mask = (s2.sign() == s1.sign()) & (s3.sign() == s1.sign()) & (s4.sign() == s1.sign())

    if mask.sum() != len(mask):
        print(mask.sum())


    return A_data[mask],A_latent[mask],x_centre[mask]

#Get the position of the grid lines in the latent space
z_1_latent_grid_pos = torch.linspace(-limLatent,limLatent,n_cells+1)
z_2_latent_grid_pos = torch.linspace(-limLatent,limLatent,n_cells+1)

A_data_list_const_jac = []
A_data_list_var_jac = []

x_centre_const_jac_list = []
x_centre_var_jac_list = []

for i in range(n_cells):
    A_data_list_const_jac.append([])
    A_data_list_var_jac.append([])

    x_centre_const_jac_list.append([])
    x_centre_var_jac_list.append([])

    for j in range(n_cells):

        A_data_const_jac,A_latent_const_jac,x_centre_const_jac = get_transformed_Area(
            z_1_lims=[z_1_latent_grid_pos[i],z_1_latent_grid_pos[i+1]],
            z_2_lims=[z_2_latent_grid_pos[j],z_2_latent_grid_pos[j+1]],
            INN=INN_const_jac,
            res=n_res_cell
            )
        
        A_data_var_jac,A_latent_var_jac,x_centre_var_jac = get_transformed_Area(
            z_1_lims=[z_1_latent_grid_pos[i],z_1_latent_grid_pos[i+1]],
            z_2_lims=[z_2_latent_grid_pos[j],z_2_latent_grid_pos[j+1]],
            INN=INN_variable_jac,
            res=n_res_cell
            )
        
        A_data_const_jac_ij = (A_data_const_jac / A_latent_const_jac).mean()
        A_data_var_jac_ij = (A_data_var_jac / A_latent_var_jac).mean()

        print(f"Area for cell {i},{j} with constant jacobian: {A_data_const_jac_ij}")
        print(f"Area for cell {i},{j} with variable jacobian: {A_data_var_jac_ij}")

        print("")
        

        A_data_list_const_jac[i].append(A_data_const_jac_ij)
        A_data_list_var_jac[i].append(A_data_var_jac_ij)

        x_centre_const_jac_list[i].append(x_centre_const_jac)
        x_centre_var_jac_list[i].append(x_centre_var_jac)

In [ ]:
res_grid_lines = 1000

z_points_grid_lines = torch.zeros([0,2])

#Vertical lines
for i,z_1_i in enumerate(z_1_latent_grid_pos):

    z_1_grid = torch.ones(res_grid_lines).reshape(-1,1) * z_1_i
    z_2_grid = torch.linspace(-limLatent,limLatent,res_grid_lines).reshape(-1,1)

    points = torch.concatenate((z_1_grid,z_2_grid),dim=1)

    z_points_grid_lines = torch.cat((z_points_grid_lines,points),dim=0)

#Horizontal lines
for i,z_2_i in enumerate(z_2_latent_grid_pos):

    z_2_grid = torch.ones(res_grid_lines).reshape(-1,1) * z_2_i
    z_1_grid = torch.linspace(-limLatent,limLatent,res_grid_lines).reshape(-1,1)

    points = torch.concatenate((z_1_grid,z_2_grid),dim=1)

    z_points_grid_lines = torch.cat((z_points_grid_lines,points),dim=0)


with torch.no_grad():
    
#Get the grid lines in data space
    x_points_grid_lines_const_jac,_ = INN_const_jac(z_points_grid_lines.to(device),rev = True)
    x_points_grid_lines_variable_jac,_ = INN_variable_jac(z_points_grid_lines.to(device),rev = True)

In [ ]:
#Evaluate the density on a grid

#Latent distribution
p_0 = force_to(D.MultivariateNormal(torch.zeros(2),torch.eye(2)),device)

def eval_INN_dist(x,INN,device):
    
    with torch.no_grad():
        #Get the latent representation
        z,jac = INN(x.to(device))

        #Get log prob
        log_prob = p_0.log_prob(z) + jac

        return log_prob.exp()
    

p_const_jac = partial(eval_INN_dist,INN = INN_const_jac,device = device)
p_variable_jac = partial(eval_INN_dist,INN = INN_variable_jac,device = device)

In [ ]:
pdf_const_jac_grid,x_grid,y_grid = eval_pdf_on_grid_2D(pdf = p_const_jac,x_lims = [-lim,lim],y_lims = [-lim,lim],x_res = res,y_res = res)
pdf_variable_jac_grid,x_grid,y_grid = eval_pdf_on_grid_2D(pdf = p_variable_jac,x_lims = [-lim,lim],y_lims = [-lim,lim],x_res = res,y_res = res)

In [ ]:
from matplotlib.colors import Normalize

fig,axes = plt.subplots(1,2,figsize=(12,4))

A_data_var_jac = torch.tensor(A_data_list_var_jac)
A_data_const_jac = torch.tensor(A_data_list_const_jac)

norm = Normalize(vmin=min(A_data_var_jac.min().item(),A_data_const_jac.min().item()), vmax = max(A_data_var_jac.max().item(),A_data_const_jac.max().item()))

cmap = "viridis"

for i in range(n_cells):
    for j in range(n_cells):

        stp = 5

        s = axes[0].scatter(
            x_centre_const_jac_list[i][j].cpu().numpy()[:,0][::stp],
            x_centre_const_jac_list[i][j].cpu().numpy()[:,1][::stp],
            c = A_data_const_jac[i,j]* torch.ones(len(x_centre_const_jac_list[i][j][::stp])),
            cmap=cmap,
            s=1,
            norm=norm
            )
        
        s = axes[1].scatter(
            x_centre_var_jac_list[i][j].cpu().numpy()[:,0][::stp],
            x_centre_var_jac_list[i][j].cpu().numpy()[:,1][::stp],
            c = A_data_var_jac[i,j]* torch.ones(len(x_centre_var_jac_list[i][j][::stp])),
            cmap=cmap,
            s=1,
            norm=norm
            )
    

#Plot the lines
axes[0].scatter(x_points_grid_lines_const_jac[:,0],x_points_grid_lines_const_jac[:,1],s=0.5,c ="k")
axes[1].scatter(x_points_grid_lines_variable_jac[:,0],x_points_grid_lines_variable_jac[:,1],s=0.5,c ="k")

#Plot the contour lines of the dnesity
n_levels = 5

axes[0].contour(x_grid,y_grid,pdf_const_jac_grid,levels = n_levels,linewidths = 1.5,colors = "w",linestyles = "dashed")
axes[1].contour(x_grid,y_grid,pdf_variable_jac_grid,levels = n_levels,linewidths = 1.5,colors = "w",linestyles = "dashed")

axes[0].set_title("volume-preserving ")
axes[1].set_title("non volume-preserving ")

for i in range(2):
    axes[i].spines['top'].set_visible(False)
    axes[i].spines['right'].set_visible(False)
    axes[i].spines['bottom'].set_visible(True)
    axes[i].spines['left'].set_visible(True)

    axes[i].set_xlabel("x")
    axes[i].set_ylabel("y")

    plt.colorbar(s,ax=axes[i])

plt.tight_layout()
plt.savefig(
    os.path.join(folder,"GMM_volume_preservation.jpeg"),
    dpi = 300
    )
plt.close()

Plot the optimal volume preserving flow:

---

construct the mapping with unit jacobian

In [ ]:
# Limit
lim = 3
grid_res = 400

# Get the latent distribution
p_0 = force_to(D.MultivariateNormal(torch.zeros(2),torch.eye(2)),device)

x_vals = torch.linspace(-lim,lim,grid_res)
y_vals = torch.linspace(-lim,lim,grid_res)

x_grid,y_grid = torch.meshgrid(x_vals,y_vals)

points = torch.cat((x_grid.reshape(-1,1),y_grid.reshape(-1,1)),1)

density_points_target = p_GMM.log_prob(points).exp()
density_points_latent = p_0.log_prob(points).exp()

def construct_data_unit_vol_change():
    """
    For a given latent distribution compute the optimal data distribution for a unit volume change.
    """

    # Get the sorting by the values of the target density
    idx_data = density_points_target.argsort(descending = True)
    points_sorted = points[idx_data]

    # Get the sorting by the values of the latent density
    idx_latent = density_points_latent.argsort(descending = True)
    density_points_latent_sorted = density_points_latent[idx_latent]

    # Arange the densty and the points on grids
    sorted_indices = np.lexsort((points_sorted[:, 1], points_sorted[:, 0]))

    points_sorted = points_sorted[sorted_indices]
    density_points_latent_sorted = density_points_latent_sorted[sorted_indices]


    x_grid = points_sorted[:,0].reshape(grid_res,grid_res)
    y_grid = points_sorted[:,1].reshape(grid_res,grid_res)

    density_points_latent_sorted = density_points_latent_sorted.reshape(grid_res,grid_res)

    return x_grid,y_grid,density_points_latent_sorted

def construct_latent_unit_vol_change():
    """
    For a given latent distribution compute the optimal data distribution for a unit volume change.
    """

    # Get the sorting by the values of the target density
    idx_latent = density_points_latent.argsort(descending = True)
    points_sorted = points[idx_latent]

    # Get the sorting by the values of the latent density
    idx_data = density_points_target.argsort(descending = True)
    density_points_data_sorted = density_points_target[idx_data]

    # Arange the densty and the points on grids
    sorted_indices = np.lexsort((points_sorted[:, 1], points_sorted[:, 0]))

    points_sorted = points_sorted[sorted_indices]
    density_points_data_sorted = density_points_data_sorted[sorted_indices]


    x_grid = points_sorted[:,0].reshape(grid_res,grid_res)
    y_grid = points_sorted[:,1].reshape(grid_res,grid_res)

    density_points_data_sorted = density_points_data_sorted.reshape(grid_res,grid_res)

    return x_grid,y_grid,density_points_data_sorted



In [ ]:
x_grid,y_grid,p_z_star = construct_latent_unit_vol_change()

dA = (x_vals[1] - x_vals[0]) * (y_vals[1] - y_vals[0])

#Approximate the covarianve matrix
cov = torch.zeros(2,2)

mean_x = (x_grid.reshape(-1) * p_z_star.reshape(-1) * dA).sum()
mean_y = (y_grid.reshape(-1) * p_z_star.reshape(-1) * dA).sum()

var_x = ((x_grid.reshape(-1) - mean_x)**2 * p_z_star.reshape(-1) * dA).sum()
var_y = ((y_grid.reshape(-1) - mean_y)**2 * p_z_star.reshape(-1) * dA).sum()
cov_xy = ((x_grid.reshape(-1) - mean_x) * (y_grid.reshape(-1) - mean_y) * p_z_star.reshape(-1) * dA).sum()

cov[0,0] = var_x
cov[1,1] = var_y
cov[0,1] = cov_xy
cov[1,0] = cov_xy

# Get the determinant of the covariance matrix
det_cov = cov[0,0] * cov[1,1] - cov[0,1] * cov[1,0]

C = det_cov**(- 1 / 4)

In [ ]:
x_grid,y_grid,p_data_star_constructed = construct_data_unit_vol_change()

p_x_star = p_data_star_constructed ** (C**2)
Z_x_star = p_x_star.sum() * dA
p_x_star = p_x_star / Z_x_star

In [ ]:
p_const_jac = partial(eval_INN_dist,INN = INN_const_jac,device = device)
p_variable_jac = partial(eval_INN_dist,INN = INN_variable_jac,device = device)

densits_points_const_jac = p_const_jac(points).cpu()
densits_points_variable_jac = p_variable_jac(points).cpu()

In [ ]:
fig,axes = plt.subplots(2,2,figsize=(13,13))

lim_x = 1.5
cmap = "jet"
s = 45
fs = 20

density_list = [
    density_points_target.reshape(grid_res,grid_res),
    p_x_star,
    densits_points_variable_jac.reshape(grid_res,grid_res),
    densits_points_const_jac.reshape(grid_res,grid_res),
]

d_min = None
d_max = None

for i in range(len(density_list)):
    if (d_min is None) or (density_list[i].min() < d_min):
        d_min = density_list[i].min()
    if (d_max is None) or (density_list[i].max() > d_max):
        d_max = density_list[i].max()

print(d_min,d_max)

titels = [
    r"$p^*(x,y)$",
    r"$p_{\text{optim}}^{\text{vp}}(x,y)$",
    r"$|f_{\theta}'| \neq const.$",
    r"$|f_{\theta}'| = const.$"
]
    

for i,ax in enumerate(axes.flatten()):
    ax.pcolormesh(x_grid,y_grid,density_list[i],cmap=cmap,vmin=d_min,vmax=d_max)
    ax.contour(x_grid,y_grid,density_list[i],levels = 5,linewidths = 1.5,colors = "k")
    
for i,ax in enumerate(axes.flatten()):
    ax.set_xlim(-lim_x,lim_x)
    ax.set_ylim(-lim_x,lim_x)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect("equal")
    ax.set_xlabel("x",fontsize=fs)
    ax.set_ylabel("y",fontsize=fs)
    ax.set_title(titels[i],fontsize=fs)

    plt.tight_layout()

plt.contour
plt.savefig(os.path.join(folder,"unit_volume_change.jpeg"),dpi=300)